In [2]:
pip install selenium beautifulsoup4 pandas webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set up Selenium WebDriver
chrome_driver_path = r"C:\Users\shras\Downloads\chromedriver-win64\chromedriver.exe" # Replace with your actual path
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Open the NSE website (Update this URL)
url = "https://www.nseindia.com/companies-listing/corporate-filings-bussiness-sustainabilitiy-reports"  # Replace with actual URL
driver.get(url)

# Wait for the table to load using its ID
try:
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "CFBussinessSustainabilitiyTable")))
    print("✅ Table loaded successfully.")
except Exception as e:
    print(f"❌ Error: Table did not load. {e}")
    driver.quit()
    exit()

# Scroll to the table (for dynamically loaded content)
table = driver.find_element(By.ID, "CFBussinessSustainabilitiyTable")
driver.execute_script("arguments[0].scrollIntoView();", table)
time.sleep(2)

# Extract data
data = []
rows = table.find_elements(By.TAG_NAME, "tr")

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")

    if len(cols) >= 7:
        company_name = cols[0].text.strip()
        fy_from = cols[1].text.strip()
        fy_to = cols[2].text.strip()

        # Handle missing links
        try:
            pdf_link = cols[3].find_element(By.TAG_NAME, "a").get_attribute("href")
        except:
            pdf_link = "No Link"

        try:
            xbrl_link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
        except:
            xbrl_link = "No Link"

        submission_date = cols[5].text.strip()
        revision_date = cols[6].text.strip()

        data.append([company_name, fy_from, fy_to, pdf_link, xbrl_link, submission_date, revision_date])

# Close Selenium session
driver.quit()

# Save data to CSV
df = pd.DataFrame(data, columns=["Company Name", "FY From", "FY To", "PDF Link", "XBRL Link", "Submission Date", "Revision Date"])
df.to_csv("nse_br_filingdata.csv", index=False)

print("✅ Scraping completed. Data saved to nse_br_filingdata.csv")


✅ Table loaded successfully.


PermissionError: [Errno 13] Permission denied: 'nse_br_filingdata.csv'

Scaling for more than 1 page data

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set up Selenium WebDriver
chrome_driver_path = r"C:\Users\shras\Downloads\chromedriver-win64\chromedriver.exe" # Replace with your actual path
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Open the NSE website (Update this URL)
url = "https://www.nseindia.com/companies-listing/corporate-filings-bussiness-sustainabilitiy-reports"  # Replace with actual URL
driver.get(url)

# Wait for the table to load using its ID
try:
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "CFBussinessSustainabilitiyTable")))
    print("✅ Table loaded successfully.")
except Exception as e:
    print(f"❌ Error: Table did not load. {e}")
    driver.quit()
    exit()

# Initialize list to store the data
data = []

# Loop through pages until there is no "Next" button
while True:
    # Scroll to the table (for dynamically loaded content)
    table = driver.find_element(By.ID, "CFBussinessSustainabilitiyTable")
    driver.execute_script("arguments[0].scrollIntoView();", table)
    time.sleep(2)

    # Extract data
    rows = table.find_elements(By.TAG_NAME, "tr")

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        
        if len(cols) >= 7:
            company_name = cols[0].text.strip()
            fy_from = cols[1].text.strip()
            fy_to = cols[2].text.strip()

            # Handle missing links
            try:
                pdf_link = cols[3].find_element(By.TAG_NAME, "a").get_attribute("href")
            except:
                pdf_link = "No Link"

            try:
                xbrl_link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
            except:
                xbrl_link = "No Link"

            submission_date = cols[5].text.strip()
            revision_date = cols[6].text.strip()

            data.append([company_name, fy_from, fy_to, pdf_link, xbrl_link, submission_date, revision_date])

    # Check for the "Next" button to load the next page
    try:
       # next_button = driver.find_element(By.XPATH, '//button[contains(text(),"Next")]')
        next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="next"]')))
        next_button.click()  # Click the "Next" button
        time.sleep(3)  # Wait for the next page to load
    except:
        print("No more pages to load.")
        break

# Close Selenium session
driver.quit()

# Save data to CSV
df = pd.DataFrame(data, columns=["Company Name", "FY From", "FY To", "PDF Link", "XBRL Link", "Submission Date", "Revision Date"])
df.to_csv("nse_br_filingdata2.csv", index=False)

print("✅ Scraping completed. Data saved to nse_br_filingdata.csv")



✅ Table loaded successfully.
No more pages to load.
✅ Scraping completed. Data saved to nse_br_filingdata.csv


Yahoo Finance

In [5]:
pip install yfinance pandas selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd

# Load the existing NSE scraped data
df = pd.read_csv("nse_br_filingdata2.csv")

# Ensure Filing Date is in proper datetime format
df["Filing Date"] = pd.to_datetime(df["Submission Date"]).dt.strftime("%Y-%m-%d")



In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_ticker_dict_batchwise(save_path='ticker_list.csv'):
    base_url = 'https://money.rediff.com/companies/nseall/'
    ticker_map = {}

    start = 1
    end = 200
    max_end = 3600  # Rediff currently has ~3571 companies

    while start <= max_end:
        url = f"{base_url}{start}-{end}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        rows = soup.select('table.dataTable tr')[1:]  # skip header
        if not rows:
            print(f"No entries found at range {start}-{end}. Ending.")
            break

        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 2:
                name = cols[0].text.strip().lower()
                ticker = cols[1].text.strip()
                ticker_map[name] = ticker

        print(f"✅ Scraped range {start}-{end} with {len(rows)} rows")
        start += 200
        end += 200
        time.sleep(1)  # be nice to the server

    # Save to CSV
    df_tickers = pd.DataFrame(ticker_map.items(), columns=['Company Name', 'Ticker'])
    df_tickers.to_csv(save_path, index=False)
    print(f"\n✔️ Saved {len(df_tickers)} tickers to '{save_path}'")

    return ticker_map



In [25]:

import difflib

# Step 1: Scrape company names & tickers from Rediff
ticker_dict = get_ticker_dict_batchwise()

# Step 2: Extract your company names from DataFrame
my_companies = df['Company Name'].dropna().tolist()

# Step 3: Fuzzy match each company name to the scraped names
def match_companies_to_tickers(companies, ticker_dict):
    matched = {}
    known_names = list(ticker_dict.keys())

    for name in companies:
        name_clean = name.lower().strip()
        matches = difflib.get_close_matches(name_clean, known_names, n=1, cutoff=0.7)
        if matches:
            matched[name] = ticker_dict[matches[0]]
        else:
            matched[name] = None  # No match found

    return matched

# Step 4: Run the matching
company_to_ticker = match_companies_to_tickers(my_companies, ticker_dict)

# Step 5: Add ticker column to your original DataFrame
df['Ticker'] = df['Company Name'].map(company_to_ticker)


✅ Scraped range 1-200 with 201 rows
✅ Scraped range 201-400 with 201 rows
✅ Scraped range 401-600 with 201 rows
✅ Scraped range 601-800 with 201 rows
✅ Scraped range 801-1000 with 201 rows
✅ Scraped range 1001-1200 with 201 rows
✅ Scraped range 1201-1400 with 201 rows
✅ Scraped range 1401-1600 with 201 rows
✅ Scraped range 1601-1800 with 201 rows
✅ Scraped range 1801-2000 with 201 rows
✅ Scraped range 2001-2200 with 201 rows
✅ Scraped range 2201-2400 with 201 rows
✅ Scraped range 2401-2600 with 201 rows
✅ Scraped range 2601-2800 with 201 rows
✅ Scraped range 2801-3000 with 201 rows
✅ Scraped range 3001-3200 with 201 rows
✅ Scraped range 3201-3400 with 201 rows
✅ Scraped range 3401-3600 with 161 rows

✔️ Saved 3559 tickers to 'ticker_list.csv'


In [26]:
df['Ticker']


0             VBL
1      CASTROLIND
2          CYIENT
3         SIEMENS
4             IGL
          ...    
145        WELENT
146    TIMETECHNO
147         KIOCL
148    ASHAPURMIN
149     SPCENETBE
Name: Ticker, Length: 150, dtype: object

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
print(df['Abnormal Return'])

0      0.030493
1      0.042338
2     -0.079641
3      0.039281
4      0.046883
5      0.085646
6           NaN
7           NaN
8     -0.075902
9     -0.006253
10    -0.009048
11     0.010366
12    -0.029502
13     0.005067
14          NaN
15    -0.055316
16    -0.006545
17     0.038845
18          NaN
19     0.026590
20    -0.033650
21     0.012154
22    -0.011472
23    -0.002026
24     0.026062
25    -0.019178
26    -0.024660
27    -0.048835
28    -0.008360
29    -0.029297
30     0.032382
31    -0.047947
32     0.006224
33    -0.016355
34    -0.028016
35     0.038274
36     0.125325
37     0.019983
38     0.013794
39    -0.029218
40    -0.015402
41    -0.034541
42     0.002974
43    -0.017288
44    -0.011174
45          NaN
46          NaN
47    -0.020186
48     0.072895
49    -0.032584
50    -0.016847
51     0.118954
52     0.009354
53     0.086648
54          NaN
55          NaN
56     0.009052
57    -0.008788
58    -0.006272
59     0.015107
60     0.005387
61     0.121096
62    -0

In [59]:
def get_4_day_return(ticker, start_date):
    if ticker is None or start_date is None:
        print("Ticker or start_date is None.")
        return None

    if not ticker.endswith(".NS") and ticker != "^NSEI":
        ticker += ".NS"

    end_date = start_date + timedelta(days=10)  # account for weekends
    print(f"\nFetching data for {ticker} from {start_date.date()} to {end_date.date()}")

    data = yf.download(ticker, start=start_date, end=end_date)
    print("Downloaded data:")
    print(data.head())

    if "Close" not in data:
        print("No 'Close' column in data.")
        return None

    data = data['Close'].dropna()
    print("Close prices:")
    print(data)

    if len(data) < 4:
        print(f"Not enough data. Found {len(data)} rows.")
        return None

    daily_returns = data.pct_change().dropna().iloc[:4]
    print("First 4 daily returns:")
    print(daily_returns)

    compounded_return = (daily_returns + 1).prod()
    print(f"Compounded return: {compounded_return}")

    return compounded_return

def compute_abnormal_return(ticker, filing_date_str):
    print(f"Received: ticker={ticker}, filing_date_str={filing_date_str}")
    
    filing_date = pd.to_datetime(filing_date_str, format="%Y-%m-%d", errors="coerce")
    print(f"Parsed filing date: {filing_date}")

    if pd.isna(ticker) or pd.isna(filing_date_str) or pd.isna(filing_date):
        print("Invalid input.")
        return None

    stock_return = get_4_day_return(ticker, filing_date)
    market_return = get_4_day_return("^NSEI", filing_date)

    print(f"Stock return: {stock_return}")
    print(f"Market return: {market_return}")

    if stock_return is not None and market_return is not None:
        abnormal_return = stock_return.values[0] - market_return.values[0]
        print(f"Abnormal return: {abnormal_return}")
        return abnormal_return
    else:
        print("Stock or market return is None.")
        return None

#compute_abnormal_return("VBL", "11-03-2025")
df['Abnormal Return'] = df.apply(
    lambda row: compute_abnormal_return(row['Ticker'], row['Filing Date']),
    axis=1
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Received: ticker=VBL, filing_date_str=2025-03-11
Parsed filing date: 2025-03-11 00:00:00

Fetching data for VBL.NS from 2025-03-11 to 2025-03-21
Downloaded data:
Price            Close        High         Low        Open    Volume
Ticker          VBL.NS      VBL.NS      VBL.NS      VBL.NS    VBL.NS
Date                                                                
2025-03-11  484.754456  487.152246  470.067942  477.361227   5774624
2025-03-12  477.910767  489.899748  475.463004  487.052359   9109922
2025-03-13  487.701752  497.043172  480.408467  483.555569   7798554
2025-03-17  506.784210  509.381829  491.098615  492.147669  10136334
2025-03-18  506.784210  506.784210  506.784210  506.784210         0
Close prices:
Ticker          VBL.NS
Date                  
2025-03-11  484.754456
2025-03-12  477.910767
2025-03-13  487.701752
2025-03-17  506.784210
2025-03-18  506.784210
2025-03-19  540.403320
2025-03-20  548.246094
First 4 daily returns:
Ticker        VBL.NS
Date                



[*********************100%***********************]  1 of 1 completed


Downloaded data:
Price              Close          High           Low          Open  \
Ticker     CASTROLIND.NS CASTROLIND.NS CASTROLIND.NS CASTROLIND.NS   
Date                                                                 
2025-02-25    212.716171    213.949473    210.004848    210.445312   
2025-02-27    213.841797    217.199126    212.216960    215.339383   
2025-02-28    208.125519    214.243115    206.148308    213.381753   
2025-03-03    212.735733    214.360570    206.539830    208.977086   
2025-03-04    217.336166    219.352539    209.564385    211.424128   

Price             Volume  
Ticker     CASTROLIND.NS  
Date                      
2025-02-25       4201668  
2025-02-27       5889216  
2025-02-28       7794409  
2025-03-03       5459323  
2025-03-04       8329827  
Close prices:
Ticker      CASTROLIND.NS
Date                     
2025-02-25     212.716171
2025-02-27     213.841797
2025-02-28     208.125519
2025-03-03     212.735733
2025-03-04     217.336166
2025-03-05

[*********************100%***********************]  1 of 1 completed


...

Downloaded data:
Price            Close        High         Low        Open   Volume
Ticker        KIOCL.NS    KIOCL.NS    KIOCL.NS    KIOCL.NS KIOCL.NS
Date                                                               
2024-09-05  407.299988  415.000000  406.750000  411.000000    40855
2024-09-06  398.649994  413.750000  396.350006  411.000000    62423
2024-09-09  391.600006  397.250000  388.950012  397.100006    57169
2024-09-10  398.600006  401.950012  394.049988  396.000000    39337
2024-09-11  391.750000  402.000000  390.000000  399.600006    36679
Close prices:
Ticker        KIOCL.NS
Date                  
2024-09-05  407.299988
2024-09-06  398.649994
2024-09-09  391.600006
2024-09-10  398.600006
2024-09-11  391.750000
2024-09-12  400.750000
2024-09-13  405.299988
First 4 daily returns:
Ticker      KIOCL.NS
Date                
2024-09-06 -0.021237
2024-09-09 -0.017685
2024-09-10  0.017875
2024-09-11 -0.017185
Compounded return: Ticker
KIOCL.NS    0.961822
dtype: float64

Fetchi

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded data:
Price              Close          High           Low          Open  \
Ticker     ASHAPURMIN.NS ASHAPURMIN.NS ASHAPURMIN.NS ASHAPURMIN.NS   
Date                                                                 
2024-09-05    317.799988    324.250000    316.850006    322.299988   
2024-09-06    311.700012    321.700012    308.799988    318.450012   
2024-09-09    291.799988    313.450012    289.100006    313.450012   
2024-09-10    306.700012    312.750000    292.000000    295.850006   
2024-09-11    304.250000    312.750000    302.500000    312.750000   

Price             Volume  
Ticker     ASHAPURMIN.NS  
Date                      
2024-09-05        183643  
2024-09-06        279028  
2024-09-09        905195  
2024-09-10        531861  
2024-09-11        218016  
Close prices:
Ticker      ASHAPURMIN.NS
Date                     
2024-09-05     317.799988
2024-09-06     311.700012
2024-09-09     291.799988
2024-09-10     306.700012
2024-09-11     304.250000
2024-09-12

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPCENETBE.NS']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed

Downloaded data:
Empty DataFrame
Columns: [(Adj Close, SPCENETBE.NS), (Close, SPCENETBE.NS), (High, SPCENETBE.NS), (Low, SPCENETBE.NS), (Open, SPCENETBE.NS), (Volume, SPCENETBE.NS)]
Index: []
Close prices:
Empty DataFrame
Columns: [SPCENETBE.NS]
Index: []
Not enough data. Found 0 rows.

Fetching data for ^NSEI from 2024-09-05 to 2024-09-15
Downloaded data:
Price              Close          High           Low          Open  Volume
Ticker             ^NSEI         ^NSEI         ^NSEI         ^NSEI   ^NSEI
Date                                                                      
2024-09-05  25145.099609  25275.449219  25127.750000  25250.500000  222200
2024-09-06  24852.150391  25168.750000  24801.300781  25093.699219  311700
2024-09-09  24936.400391  24957.500000  24753.150391  24823.400391  254400
2024-09-10  25041.099609  25130.500000  24896.800781  24999.400391  251300
2024-09-11  24918.449219  25113.699219  24885.150391  25034.000000  279200
Close prices:
Ticker             ^NSEI
Da

In [62]:
df.to_csv("data_with_abnormal_returns.csv", index=False)

In [65]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.Collecting pymupdf
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB 393.8 kB/s eta 0:00:42
   ---------------------------------------- 0.0/16.6 MB 393.8 kB/s eta 0:00:42
   ---------------------------------------- 0.1/16.6 MB 357.2 kB/s eta 0:00:47
   ---------------------------------------- 0.1/16.6 MB 403.5 kB/s eta 0:00:41
   ---------------------------------------- 0.1/16.6 MB 363.1 kB/s eta 0:00:46
   ---------------------------------------- 0.1/16.6 MB 363.1 kB/s eta 0:00:46
   ---------------------------------------- 0.1/16.6 MB 387.0 kB/s eta 0:00:43
   ---------------------------------------- 0.2/16.6 MB 402.6 kB/s eta 0:00:41
   ---------------------------------------- 0.2/16.6 MB 402.6 kB/s eta 0:0


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
import requests
import fitz  # PyMuPDF

def download_pdf(url, output_file="downloaded_pdf.pdf"):
    # Download the PDF file from the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        with open(output_file, 'wb') as file:
            file.write(response.content)
        print(f"PDF downloaded successfully: {output_file}")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

def extract_text_from_pdf(pdf_url, chunk_size=50, output_file="extracted_text.txt"):
    # First, download the PDF
    downloaded_pdf = "downloaded_pdf.pdf"
    download_pdf(pdf_url, downloaded_pdf)
    
    # Now, extract text from the downloaded PDF
    doc = fitz.open(downloaded_pdf)
    with open(output_file, 'w', encoding='utf-8') as output:
        total_pages = doc.page_count
        for start_page in range(0, total_pages, chunk_size):
            end_page = min(start_page + chunk_size, total_pages)
            # Process pages from start_page to end_page
            for page_num in range(start_page, end_page):
                page = doc.load_page(page_num)
                text = page.get_text()
                output.write(f"Page {page_num + 1}:\n")
                output.write(text)
                output.write("\n" + "="*50 + "\n")
    
    print(f"Text extraction completed in chunks. Output saved to {output_file}.")

# Example usage
pdf_url = 'https://nsearchives.nseindia.com/corporate/VBL_11032025174358_VBLNoticeAR2024.pdf'
extract_text_from_pdf(pdf_url)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))